## Table of Contents

1. [Introduction](#1.-introduction)

2. [Imports](#2.-imports)

3. [Daily Return Preparation](#3.-daily-return-preparation)

   3.1 [Load Input CSV files](#3.1-load-input-csv-files)

   3.2 [Clean & Standardize Price Data](#3.2-clean-&-standardize-price-data)

   3.3 [Compute Daily Returns $R_i$, $log(R_i)$, $R_m$, $log(R_m)$, and $R_f$](#3.3-compute-daily-returns-r_i-logr_i-r_m-logr_m-and-r_f)

   3.4 [Align and Merge $R_i$](#3.4-align-and-merge-r_i)

   3.5 [Save Outputs](#3.5-save-outputs)

### 1. Introduction

### 2. Imports

In [23]:
import pandas as pd
import numpy as np

### 3. Daily Return Preparation

##### 3.1 Load Input CSV files

In [24]:
# Individual stocks
tickers = ["jpm", "bac", "wfc", "gs", "blk", "pld", "amt", "spg", "avb", "o"]
stock_data = {}

for t in tickers:
    stock_data[t.upper()] = pd.read_csv(f"../data/in/{t}.csv")
    print(t.upper(), stock_data[t.upper()].shape)

# Market index
sp500 = pd.read_csv("../data/in/sp500.csv")
print("\nS&P 500 shape:", sp500.shape)

# Risk-free rate
tbill = pd.read_csv("../data/in/13w_tbill.csv")
print("\nT-bill shape:", tbill.shape)

JPM (1256, 6)
BAC (1256, 6)
WFC (1256, 6)
GS (1256, 6)
BLK (1256, 6)
PLD (1256, 6)
AMT (1256, 6)
SPG (1256, 6)
AVB (1256, 6)
O (1256, 6)

S&P 500 shape: (1255, 6)

T-bill shape: (1256, 6)


##### 3.2 Clean & Standardize Price Data

In [25]:
# Clean individual stock price files
clean_stocks = {}

for t in tickers:
    df = stock_data[t.upper()].copy()

    # Convert Date to datetime
    df['Date'] = pd.to_datetime(df['Date'])

    # Clean "Close/Last" (remove any "$" symbols, convert to float)
    df['Close'] = df['Close/Last'].replace('[\\$,]', '', regex=True).astype(float)

    # Keep only Date + Close columns
    df = df[['Date', 'Close']].sort_values('Date')

    clean_stocks[t.upper()] = df

    print(f"Sample cleaned stock ({t.upper()}):")
    print(clean_stocks[t.upper()].head(3))

# Clean S&P 500
sp500_clean = sp500.copy()
sp500_clean['Date'] = pd.to_datetime(sp500_clean['Date'], format='%m/%d/%y')
sp500_clean = sp500_clean[['Date', 'Close']].sort_values('Date')

# Clean 13-week T-bill
tbill_clean = tbill.copy()
tbill_clean['Date'] = pd.to_datetime(tbill_clean['Date'], format='%m/%d/%y')
tbill_clean = tbill_clean[['Date', 'Close']].sort_values('Date')

# Show first 3 rows from each for verification
print("Sample cleaned stock (JPM):")
print(clean_stocks['JPM'].head(3))

print("\nSample cleaned S&P500:")
print(sp500_clean.head(3))

print("\nSample cleaned T-bill:")
print(tbill_clean.head(3))

Sample cleaned stock (JPM):
           Date   Close
1255 2020-10-19   99.80
1254 2020-10-20  100.37
1253 2020-10-21   99.37
Sample cleaned stock (BAC):
           Date  Close
1255 2020-10-19  23.72
1254 2020-10-20  24.14
1253 2020-10-21  24.05
Sample cleaned stock (WFC):
           Date  Close
1255 2020-10-19  22.54
1254 2020-10-20  22.81
1253 2020-10-21  22.70
Sample cleaned stock (GS):
           Date   Close
1255 2020-10-19  205.69
1254 2020-10-20  208.03
1253 2020-10-21  202.91
Sample cleaned stock (BLK):
           Date   Close
1255 2020-10-19  634.89
1254 2020-10-20  645.07
1253 2020-10-21  634.99
Sample cleaned stock (PLD):
           Date   Close
1255 2020-10-19  103.02
1254 2020-10-20  103.53
1253 2020-10-21  102.65
Sample cleaned stock (AMT):
           Date   Close
1255 2020-10-19  239.60
1254 2020-10-20  243.80
1253 2020-10-21  241.74
Sample cleaned stock (SPG):
           Date  Close
1255 2020-10-19  63.91
1254 2020-10-20  64.46
1253 2020-10-21  64.21
Sample cleaned stock 

##### 3.3 Compute Daily Returns $R_i$, $log(R_i)$, $R_m$, $log(R_m)$, and $R_f$

In [26]:
# Compute daily returns (Ret) and log returns (LogRet) for each stock
stock_returns = {}

for t in tickers:
    df = clean_stocks[t.upper()].copy()

    # Arithmetic return
    df['Ret'] = df['Close'] / df['Close'].shift(1) - 1

    # Log return
    df['LogRet'] = np.log(df['Close'] / df['Close'].shift(1))

    df = df.dropna()
    stock_returns[t.upper()] = df[['Date', 'Ret', 'LogRet']]
    
    print(f"Sample Returns (Ri = {t.upper()}):")
    print(stock_returns[t.upper()].head(3))

# Compute market returns and log returns
sp500_ret = sp500_clean.copy()
sp500_ret['Market_Ret'] = sp500_ret['Close'] / sp500_ret['Close'].shift(1) - 1
sp500_ret['Market_LogRet'] = np.log(sp500_ret['Close'] / sp500_ret['Close'].shift(1))
sp500_ret = sp500_ret.dropna()
sp500_ret = sp500_ret[['Date', 'Market_Ret', 'Market_LogRet']]

print(f"Sample Market Returns (Rm):")
print(sp500_ret.head(3))

# Convert T-bill annualized yield to daily rate (RF)
tbill_daily = tbill_clean.copy()
tbill_daily['RF_Daily'] = (1 + tbill_daily['Close']/100)**(1/252) - 1
tbill_daily = tbill_daily[['Date', 'RF_Daily']]

print(f"Sample Daily Risk-Free Rate (Rf):")
print(tbill_daily.head(3))

Sample Returns (Ri = JPM):
           Date       Ret    LogRet
1254 2020-10-20  0.005711  0.005695
1253 2020-10-21 -0.009963 -0.010013
1252 2020-10-22  0.035323  0.034713
Sample Returns (Ri = BAC):
           Date       Ret    LogRet
1254 2020-10-20  0.017707  0.017552
1253 2020-10-21 -0.003728 -0.003735
1252 2020-10-22  0.034096  0.033527
Sample Returns (Ri = WFC):
           Date       Ret    LogRet
1254 2020-10-20  0.011979  0.011908
1253 2020-10-21 -0.004822 -0.004834
1252 2020-10-22  0.024229  0.023940
Sample Returns (Ri = GS):
           Date       Ret    LogRet
1254 2020-10-20  0.011376  0.011312
1253 2020-10-21 -0.024612 -0.024920
1252 2020-10-22  0.012271  0.012197
Sample Returns (Ri = BLK):
           Date       Ret    LogRet
1254 2020-10-20  0.016034  0.015907
1253 2020-10-21 -0.015626 -0.015750
1252 2020-10-22  0.000299  0.000299
Sample Returns (Ri = PLD):
           Date       Ret    LogRet
1254 2020-10-20  0.004950  0.004938
1253 2020-10-21 -0.008500 -0.008536
1252 2020-1

##### 3.4 Align and Merge $R_i$

In [27]:
# Merge all stock returns into wide format

merged_stocks = None

for t in tickers:
    df = stock_returns[t.upper()].copy()
    df = df.rename(columns={
        'Ret': f"{t.upper()}_Ret",
        'LogRet': f"{t.upper()}_LogRet"
    })

    if merged_stocks is None:
        merged_stocks = df
    else:
        merged_stocks = pd.merge(merged_stocks, df, on='Date', how='inner')

print("Merged Daily Stock Returns (wide format):")
print(merged_stocks.head(3))

Merged Daily Stock Returns (wide format):
        Date   JPM_Ret  JPM_LogRet   BAC_Ret  BAC_LogRet   WFC_Ret  \
0 2020-10-20  0.005711    0.005695  0.017707    0.017552  0.011979   
1 2020-10-21 -0.009963   -0.010013 -0.003728   -0.003735 -0.004822   
2 2020-10-22  0.035323    0.034713  0.034096    0.033527  0.024229   

   WFC_LogRet    GS_Ret  GS_LogRet   BLK_Ret  ...   PLD_Ret  PLD_LogRet  \
0    0.011908  0.011376   0.011312  0.016034  ...  0.004950    0.004938   
1   -0.004834 -0.024612  -0.024920 -0.015626  ... -0.008500   -0.008536   
2    0.023940  0.012271   0.012197  0.000299  ...  0.001461    0.001460   

    AMT_Ret  AMT_LogRet   SPG_Ret  SPG_LogRet   AVB_Ret  AVB_LogRet     O_Ret  \
0  0.017529    0.017377  0.008606    0.008569  0.000963    0.000962  0.005957   
1 -0.008450   -0.008485 -0.003878   -0.003886  0.002886    0.002881  0.009814   
2 -0.024696   -0.025006  0.033328    0.032785  0.003494    0.003488  0.006870   

   O_LogRet  
0  0.005940  
1  0.009766  
2  0.0068

##### 3.5 Save Outputs

In [28]:
# Save wide stock returns
merged_stocks.to_csv("../data/out/daily_stock_returns.csv", index=False)
print("Saved: daily_stock_returns.csv")

# Save market returns
daily_market_returns = sp500_ret.copy()
daily_market_returns.to_csv("../data/out/daily_market_returns.csv", index=False)
print("Saved: daily_market_returns.csv")

# Save risk-free returns
daily_riskfree_returns = tbill_daily.copy()
daily_riskfree_returns.to_csv("../data/out/daily_riskfree_returns.csv", index=False)
print("Saved: daily_riskfree_returns.csv")

Saved: daily_stock_returns.csv
Saved: daily_market_returns.csv
Saved: daily_riskfree_returns.csv
